In [143]:
%matplotlib inline
import geopandas as gpd
import pandas as pd
import folium

In [20]:
fema = gpd.read_file("Data/FEMA_6/FEMA_6.shp")
fema = fema.sort_values(by='CTFIPS')

In [228]:
def ag_count(state):
    data = gpd.read_file("Data/SoVI2010_" + state + "/SoVI0610_" + state + ".shp")
    data['CTFIPS'] = data['STATEFP10'] + data['COUNTYFP10']
    data = data.groupby('CTFIPS').agg({"SOVI0610" + state: "mean", "P0010001": "sum"})
    data = fema.merge(data, on="CTFIPS", how="right")
    data = data.rename(columns={
        "SOVI0610" + state: "SOVI",
        "P0010001": "POPULATION",
        "TOTAL_DECL": "TOTAL_DECLARATIONS"
    })
    data = data.sort_values(by='CTFIPS')
    data['SOVI_RANK'] = data['SOVI'].rank()
    rank = []
    n = len(data)    
    for index, row in data.iterrows():
        if row.SOVI_RANK < (n/3):
            rank.append(1)
        elif row.SOVI_RANK < (2 * (n/3)):
            rank.append(2)
        else:
            rank.append(3)
    data['RANK'] = rank
    return data

CA = ag_count("CA")
FL = ag_count("FL")
IN = ag_count("IN")
LA = ag_count("LA")
ME = ag_count("ME")
WA = ag_count("WA")
WA.head()

,OBJECTID,STFIPS,ST_ABBR,CTFIPS,STATE,COUNTY,GEO_SUFFIX,TOTAL_DECLARATIONS,FLOOD_1964,ICE_STORM_,...,REGION_NO,Shape_Leng,Winter_196,Shape_Le_1,Shape_Area,geometry,SOVI,POPULATION,SOVI_RANK,RANK
27,234,53,WA,53001,Washington,Adams,County,4,1,0,...,10,3.873694,0,3.873694,0.590971,POLYGON ((-117.9601429998994 47.26044899992155...,1.243368,18728.0,26.0,3
9,216,53,WA,53003,Washington,Asotin,County,7,4,0,...,10,2.128862,0,2.128862,0.193368,POLYGON ((-117.0398130003695 46.42542499976412...,1.508957,21623.0,29.0,3
21,228,53,WA,53005,Washington,Benton,County,11,7,0,...,10,3.260064,0,3.260064,0.531704,POLYGON ((-119.4532199996032 46.67924299996469...,-0.621206,175177.0,5.0,1
1,208,53,WA,53007,Washington,Chelan,County,12,3,0,...,10,6.258858,0,6.258858,0.932193,POLYGON ((-120.7015569995556 48.53161999976345...,0.028135,72453.0,12.0,1
7,214,53,WA,53009,Washington,Clallam,County,17,5,0,...,10,6.125295,0,6.125295,0.551389,(POLYGON ((-124.6354293765643 47.8814773622907...,2.293236,71404.0,35.0,3


In [230]:
sovi = CA.append([FL, IN, LA, ME, WA])
sovi = sovi.sort_values(by='CTFIPS')
sovi.head()

,OBJECTID,STFIPS,ST_ABBR,CTFIPS,STATE,COUNTY,GEO_SUFFIX,TOTAL_DECLARATIONS,FLOOD_1964,ICE_STORM_,...,REGION_NO,Shape_Leng,Winter_196,Shape_Le_1,Shape_Area,geometry,SOVI,POPULATION,SOVI_RANK,RANK
44,58,06,CA,06001,California,Alameda,County,14,3,0,...,9,3.610286,1,3.610286,0.196849,(POLYGON ((-121.9673689779958 37.4676361297038...,-0.174812,1510271.0,19.0,1
25,39,06,CA,06003,California,Alpine,County,6,3,0,...,9,2.174212,0,2.174212,0.199044,POLYGON ((-119.5854370003171 38.71321199963438...,0.686160,1175.0,27.0,2
7,21,06,CA,06005,California,Amador,County,9,3,0,...,9,2.909742,0,2.909742,0.161954,POLYGON ((-120.0723920002904 38.70276699999727...,1.558083,38091.0,42.0,3
39,53,06,CA,06007,California,Butte,County,10,3,0,...,9,4.205838,1,4.205838,0.455942,POLYGON ((-121.0766949997742 39.59726399983401...,1.954188,220000.0,48.0,3
8,22,06,CA,06009,California,Calaveras,County,8,1,0,...,9,2.963056,0,2.963056,0.276236,POLYGON ((-120.0199510002383 38.43352099969468...,1.929986,45578.0,47.0,3


In [221]:
fema_geo = fema[['CTFIPS', 'geometry']]
fema_geo.head()

,CTFIPS,geometry
0,06059,(POLYGON ((-117.888496211331 33.60864264354331...
1,06103,POLYGON ((-121.4976349996553 40.44559100012486...
2,06011,POLYGON ((-121.8900129998999 39.38386400012371...
3,06083,(POLYGON ((-119.0462190444732 33.4651337111520...
4,06051,"POLYGON ((-119.3283979999676 38.5347919996513,..."


In [236]:
map = folium.Map(location=[40, -100], zoom_start=3)    

map.choropleth(
    geo_data = sovi,
    name = "SOVI",
    data = sovi,
    columns = ['CTFIPS', 'RANK'],
    key_on = 'feature.properties.CTFIPS',
    fill_color = "RdYlGn",
    fill_opacity = 1.0
)

map.choropleth(
    geo_data = sovi,
    name = "FEMA",
    data = sovi,
    columns = ['CTFIPS', 'TOTAL_DECLARATIONS'],
    key_on = 'feature.properties.CTFIPS',
    fill_color = "RdYlGn",
    fill_opacity = 1.0,
    show = False
)

folium.LayerControl(collapsed=False).add_to(map)

In [237]:
map.save("map.html")